In [1]:
import pandas as pd
import numpy as np
import warnings
import warnings
from datetime import datetime
warnings.filterwarnings("ignore")

# func

In [693]:
def preprocess_tp_data(path):
  tp_data = pd.read_csv(path)
  tp_data.drop_duplicates(inplace = True)
  tp_data.last_r.loc[tp_data.last_r < 0] = -1
  tp_data.state_close_estimate.fillna(tp_data.last_r , inplace = True)
  tp_data.state_max_r.fillna(tp_data.last_r , inplace = True)
  tp_data.state_MA_r.fillna(tp_data.last_r, inplace = True)
  tp_data.state_RSI_r.fillna(tp_data.last_r, inplace = True)
  tp_data.state_MA_r.loc[tp_data.state_MA_r < -1] = -1
  tp_data.state_max_r.loc[tp_data.state_max_r < -1] = -1
  tp_data.state_chain.loc[~tp_data.state_chain.isna()] = tp_data.state_chain.loc[~tp_data.state_chain.isna()].apply(lambda x:str(x).strip('][').split(', '))
  tp_data.state_chain.loc[~tp_data.state_chain.isna()] = tp_data.state_chain.loc[~tp_data.state_chain.isna()].apply(lambda x:[float(element) for element in x])
  tp_data['state_day_start'] = tp_data['state_day_start'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
  tp_data['state_day_end'] = tp_data['state_day_end'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))

  spread_df = pd.read_csv('/Users/tranthong/Desktop/stock-app/spread.csv')
  spread_df['deviation'] = 0
  spread_df['spread'] = spread_df['spread'] / 2
  tp_data['spread'] = tp_data.state_ticker.apply(lambda x: spread_df.spread.loc[spread_df.ticker == x].iloc[0])
  tp_data['deviation'] = tp_data.state_ticker.apply(lambda x: spread_df.deviation.loc[spread_df.ticker == x].iloc[0])
  tp_data['point'] = tp_data.state_ticker.apply(lambda x: spread_df.point.loc[spread_df.ticker == x].iloc[0])
  tp_data['sum_gap'] = ((tp_data['spread'] + tp_data['deviation']) * tp_data['point']) / abs(tp_data['state_open'] - tp_data['state_stop_loss'] ) + 1
  tp_data['state_chain_new'] = [np.array(y) / x for x,y in tp_data[['sum_gap' , 'state_chain']].to_numpy()]
  return tp_data
tp_data = preprocess_tp_data('/Users/tranthong/Desktop/stock-app/state_output_new/state_output_new/stateoutput120.csv')

In [511]:
def get_return_with_max_estimate(state_return_list, min_thresh = 5, cut_thresh = 0.7):
    c = np.array(state_return_list).copy()
    x_value = c[c > min_thresh]

    if len(x_value) > 0:
        x_value = x_value[0]
        x_index, = np.where(c == x_value)
        x = c[x_index[0]:]
        list_temp = [x[0] * cut_thresh]
        for i in range(1, len(x)):
            temp = x[0:i].max() * cut_thresh
            list_temp.append(temp)
        temp_df = pd.DataFrame()
        temp_df['a'] = x
        temp_df['b'] = list_temp
        return temp_df.a.iloc[-1] if temp_df.loc[temp_df.a < temp_df.b].empty else  temp_df.loc[temp_df.a < temp_df.b].iloc[0].b
    else:
        lost_r = c[-1] if c[-1] > 0 else -1
        return lost_r


In [ ]:
def get_stock_data(tp_data):
    tp_data['estimate_with_thresh'] = tp_data.state_chain_new.apply(lambda x : get_return_with_max_estimate(x , 15 , 0.6) )
    tp_data['estimate_with_thresh2'] = tp_data.state_chain_new.apply(lambda x : x.max())
    tp_data.estimate_with_thresh2.loc[tp_data.estimate_with_thresh2 < 5] = -1
    stock_tp_data = tp_data[['state_ticker','state_close_estimate','state_max_r','state_MA_r','state_RSI_r','last_r', 'estimate_with_thresh', 'estimate_with_thresh2']].groupby('state_ticker').sum()
    stock_tp_data['win_count'] = tp_data[['state_ticker','state_close_estimate','state_max_r','state_MA_r','state_RSI_r','last_r', 'estimate_with_thresh', 'estimate_with_thresh2']].loc[tp_data.state_MA_r > 0].groupby('state_ticker').count().state_MA_r
    stock_tp_data['loss_count'] = tp_data[['state_ticker','state_close_estimate','state_max_r','state_MA_r','state_RSI_r','last_r' , 'estimate_with_thresh', 'estimate_with_thresh2']].loc[tp_data.state_MA_r < 0].groupby('state_ticker').count().state_MA_r
    stock_tp_data.win_count.fillna(0 , inplace = True)
    stock_tp_data['win_count'] = stock_tp_data['win_count'].astype(int) 

    return stock_tp_data

# TP data

In [188]:
tp_data = pd.read_csv('/Users/tranthong/Desktop/stock-app/state_output_new/state_output_new/stateoutput270.csv')
tp_data.head(5)

,state_ticker,state_order,state_day_start,state_day_end,state_open,state_stop_loss,state_close_estimate,state_max_r,state_MA_r,state_RSI_r,last_r,state_chain
0,MRK,1,2022-05-16 13:50:00,2022-05-18 13:50:00,90.985,90.510,NaN,7.400000,3.610526,3.884211,3.610526,"[-0.13684210526312074, -0.26315789473680745, -..."
1,MRK,2,2022-05-18 18:50:00,2022-05-19 19:00:00,92.204,92.394,NaN,5.915789,NaN,NaN,-1.000000,"[0.2842105263158036, 0.6526315789474258, 1.178..."
2,MRK,1,2022-05-25 16:10:00,2022-05-26 14:20:00,93.138,92.818,NaN,2.225000,0.131250,NaN,-1.000000,"[0.44374999999999665, 0.4750000000000133, 0.13..."
3,MRK,2,2022-05-26 14:35:00,2022-06-01 19:35:00,93.028,93.458,NaN,6.902326,3.600000,5.506977,3.600000,"[-0.05116279069765351, -0.5860465116278868, -0..."
4,MRK,2,2022-06-01 19:30:00,2022-06-03 13:40:00,91.385,92.145,NaN,3.480263,-0.125000,NaN,1.388158,"[-0.006578947368433847, -0.1250000000000187, 0..."


In [374]:
a = None
if a:
    print('a')

In [371]:
data1 = pd.read_csv('/Users/tranthong/Desktop/stock-app/spread.csv')
data2 = pd.read_csv('/Users/tranthong/Desktop/stock-app/spread2.csv')
data1['spread2'] = data1.ticker.apply(lambda x: data2.spread.loc[data2.ticker == x].iloc[0])
data1.loc[data1.spread != data1.spread2]

,Unnamed: 0,ticker,point,spread,deviation,spread2
0,0,AAPL,0.01,6,5,9
5,5,AMD,0.01,10,5,25
8,8,AMZN,0.01,4,5,6
12,12,BAC,0.01,8,5,9
13,13,BA,0.01,24,5,46
24,24,DE30,0.10,39,5,36
28,28,FB,0.01,8,5,41
38,38,JP225,0.10,90,5,89
50,50,MSFT,0.01,10,5,21
52,52,NFLX,0.01,24,5,81


In [259]:
tp_data = preprocess_tp_data('/Users/tranthong/Desktop/stock-app/state_output_new/state_output_new/stateoutput90.csv')
tp_data[['state_close_estimate','state_max_r','state_MA_r','state_RSI_r','last_r']].describe()

,state_close_estimate,state_max_r,state_MA_r,state_RSI_r,last_r
count,380.000000,380.000000,380.000000,380.000000,380.000000
mean,0.123399,2.594735,-0.024286,0.055182,-0.042620
std,3.929826,6.020758,3.659371,3.279577,3.668570
min,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
25%,-1.000000,-0.061935,-1.000000,-1.000000,-1.000000
50%,-1.000000,0.466667,-1.000000,-1.000000,-1.000000
75%,-1.000000,2.452899,-1.000000,-1.000000,-1.000000
max,32.342268,47.935616,32.342268,33.528571,32.342268


In [260]:
tp_data[['state_close_estimate','state_max_r','state_MA_r','state_RSI_r','last_r']].sum()

state_close_estimate     46.891751
state_max_r             985.999242
state_MA_r               -9.228510
state_RSI_r              20.969008
last_r                  -16.195593
dtype: float64

In [266]:
tp_data['estimate_with_thresh'] = tp_data.state_chain_new.apply(lambda x : get_return_with_max_estimate(x , 12 , 0.7) )
tp_data.estimate_with_thresh.sum()

-63.334356909509566

In [264]:
tp_data.head(5)

,state_ticker,state_order,state_day_start,state_day_end,state_open,state_stop_loss,state_close_estimate,state_max_r,state_MA_r,state_RSI_r,last_r,state_chain,spread,deviation,point,sum_gap,state_chain_new
0,MRK,1,2022-11-09 14:55:00,2022-11-09 19:50:00,102.024,101.814,0.885714,7.504762,0.885714,0.885714,0.885714,"[1.5999999999999324, 3.1238095238094337, 3.552...",7,5,0.01,1.571429,"[1.0181818181817892, 1.9878787878787578, 2.260..."
1,MRK,1,2022-11-16 14:55:00,2022-11-16 15:00:00,99.946,99.656,-1.000000,-0.572414,-1.000000,-1.000000,-1.000000,"[-0.572413793103425, -0.882758620689637]",7,5,0.01,1.413793,"[-0.4048780487804681, -0.6243902439024211]"
2,MRK,2,2022-11-16 19:35:00,2022-11-16 19:55:00,99.898,100.028,-1.000000,0.984615,-1.000000,-1.000000,-1.000000,"[0.9846153846153123, 0.75384615384609, 0.52307...",7,5,0.01,1.923077,"[0.5119999999999807, 0.39199999999998075, 0.27..."
3,MRK,1,2022-11-25 15:30:00,2022-11-25 15:40:00,107.268,107.076,-1.000000,-0.458333,-1.000000,-1.000000,-1.000000,"[-0.770833333333284, -0.45833333333328397, -0....",7,5,0.01,1.625000,"[-0.4743589743589509, -0.2820512820512558, -0...."
4,MRK,1,2022-11-29 18:25:00,2022-11-29 18:55:00,108.816,108.592,-1.000000,0.241071,-1.000000,-1.000000,-1.000000,"[0.10714285714285941, -0.42857142857143765, -0...",7,5,0.01,1.535714,"[0.069767441860467, -0.279069767441868, -0.046..."


# concat data

In [694]:
import glob , os
concat_data = pd.DataFrame()
for path in glob.glob("/Users/tranthong/Desktop/stock-app/state_output_new/state_output_new/*.csv"):
    tp_data = preprocess_tp_data(path)
    tp_data['bash_path'] = str(os.path.basename(path))
    concat_data = pd.concat([tp_data, concat_data])
concat_data.reset_index(inplace=True)


In [695]:
concat_data['estimate_with_thresh'] = concat_data.state_chain_new.apply(lambda x : get_return_with_max_estimate(x , 8 , 0.6) )
concat_data['estimate_with_thresh2'] = concat_data.state_chain_new.apply(lambda x : x.max() * 0.6 )
concat_data.estimate_with_thresh2.loc[concat_data.estimate_with_thresh2 < 3] = -1
concat_data.estimate_with_thresh.sum()

-222.71900448031914

In [696]:
concat_data

,index,state_ticker,state_order,state_day_start,state_day_end,state_open,state_stop_loss,state_close_estimate,state_max_r,state_MA_r,...,last_r,state_chain,spread,deviation,point,sum_gap,state_chain_new,bash_path,estimate_with_thresh,estimate_with_thresh2
0,0,MRK,1,2022-10-10 19:35:00,2022-10-11 13:40:00,90.712,90.482,-1.0000,0.382609,-1.0000,...,-1.0000,"[0.38260869565216427, -0.6173913043478976, -0....",3.5,0,0.01,1.152174,"[0.3320754716981056, -0.5358490566038369, -0.3...",stateoutput120.csv,-1.000000,-1.0
1,1,MRK,2,2022-10-11 19:05:00,2022-10-11 19:20:00,91.072,91.192,0.1000,1.100000,0.1000,...,0.1000,"[0.9333333333333728, 1.1, -0.3166666666666272,...",3.5,0,0.01,1.291667,"[0.7225806451613077, 0.8516129032257911, -0.24...",stateoutput120.csv,0.077419,-1.0
2,2,MRK,1,2022-10-14 14:00:00,2022-10-14 14:40:00,92.362,92.082,-1.0000,1.278571,-1.0000,...,-1.0000,"[1.278571428571438, 0.6000000000000203, -0.114...",3.5,0,0.01,1.125000,"[1.1365079365079453, 0.5333333333333515, -0.10...",stateoutput120.csv,-1.000000,-1.0
3,3,MRK,1,2022-10-18 13:40:00,2022-10-18 13:45:00,94.194,94.034,0.6625,0.662500,0.6625,...,0.6625,"[-0.7124999999999789, 0.6624999999999212]",3.5,0,0.01,1.218750,"[-0.5846153846153651, 0.5435897435896768]",stateoutput120.csv,0.543590,-1.0
4,4,MRK,2,2022-10-19 19:00:00,2022-10-21 13:40:00,93.268,93.448,-1.0000,4.377778,-1.0000,...,-1.0000,"[0.48888888888883625, 0.37777777777775146, 0.2...",3.5,0,0.01,1.194444,"[0.40930232558134855, 0.3162790697674177, 0.22...",stateoutput120.csv,-1.000000,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4730,681,CSCO,2,2022-05-13 18:10:00,2022-05-13 18:50:00,48.912,49.082,-1.0000,0.305882,-1.0000,...,-1.0000,"[0.3058823529411838, -0.16470588235293873, 0.1...",3.5,0,0.01,1.205882,"[0.2536585365853724, -0.13658536585365674, 0.1...",stateoutput270.csv,-1.000000,-1.0
4731,682,CSCO,2,2022-05-31 15:50:00,2022-05-31 17:35:00,45.000,45.350,-1.0000,0.028571,-1.0000,...,-1.0000,"[-0.14285714285713416, -0.4571428571428455, -0...",3.5,0,0.01,1.100000,"[-0.129870129870122, -0.4155844155844052, -0.5...",stateoutput270.csv,-1.000000,-1.0
4732,683,CSCO,1,2022-06-01 18:55:00,2022-06-02 13:40:00,45.222,45.092,-1.0000,1.138462,-1.0000,...,-1.0000,"[0.29230769230767717, 0.21538461538461287, 0.4...",3.5,0,0.01,1.269231,"[0.23030303030301666, 0.16969696969696646, 0.3...",stateoutput270.csv,-1.000000,-1.0
4733,684,CSCO,1,2022-06-06 15:15:00,2022-06-06 16:05:00,45.619,45.284,-1.0000,-0.355224,-1.0000,...,-1.0000,"[-0.3552238805970133, -0.5940298507462614, -0....",3.5,0,0.01,1.104478,"[-0.3216216216216203, -0.5378378378378315, -0....",stateoutput270.csv,-1.000000,-1.0


In [697]:
stock_tp_data = concat_data[['state_ticker','state_close_estimate','state_max_r','state_MA_r','state_RSI_r','last_r', 'estimate_with_thresh', 'estimate_with_thresh2', 'bash_path']].groupby(['state_ticker' , 'bash_path']).sum()
stock_tp_data['win_count'] = concat_data[['state_ticker','state_close_estimate','state_max_r','state_MA_r','state_RSI_r','last_r', 'estimate_with_thresh', 'estimate_with_thresh2', 'bash_path']].loc[concat_data.state_MA_r > 0].groupby(['state_ticker' , 'bash_path']).count().state_MA_r
stock_tp_data['loss_count'] = concat_data[['state_ticker','state_close_estimate','state_max_r','state_MA_r','state_RSI_r','last_r', 'estimate_with_thresh', 'estimate_with_thresh2', 'bash_path']].loc[concat_data.state_MA_r < 0].groupby(['state_ticker' , 'bash_path']).count().state_MA_r
stock_tp_data.win_count.fillna(0 , inplace = True)
stock_tp_data['win_count'] = stock_tp_data['win_count'].astype(int) 
stock_tp_data

state_close_estimate  state_max_r  \
state_ticker bash_path                                               
AAPL         stateoutput120.csv             -0.473267    24.204790   
             stateoutput150.csv             -2.117733    21.692807   
             stateoutput180.csv             -4.709986    58.150310   
             stateoutput210.csv            -13.768421    27.340992   
             stateoutput240.csv             -0.868186    16.412262   
...                                               ...          ...   
TSLA         stateoutput150.csv             -5.280309    22.691570   
             stateoutput180.csv            -11.494241     9.367959   
             stateoutput210.csv              0.842688    10.829510   
             stateoutput270.csv             -7.745189    17.497227   
             stateoutput60.csv              31.526471    63.004396   

                                 state_MA_r  state_RSI_r     last_r  \
state_ticker bash_path                                                
AAPL         stateoutput120.csv   -0.473267     2.783911  -0.473267   
             stateoutput150.csv   -1.678667    -2.117733  -2.117733   
             stateoutput180.csv    3.864214     2.939569   2.939569   
             stateoutput210.csv  -11.968672   -10.012171 -13.768421   
             stateoutput240.csv   -4.610311    -4.981395  -4.888372   
...                                     ...          ...        ...   
TSLA         stateoutput150.csv   -4.400309    -2.518035  -5.280309   
             stateoutput180.csv  -11.494241   -11.494241 -11.494241   
             stateoutput210.csv    0.842688    -2.496393   0.842688   
             stateoutput270.csv  -13.664435    -6.614169 -13.105251   
             stateoutput60.csv    31.526471    -3.120588  31.526471   

                                 estimate_with_thresh  estimate_with_thresh2  \
state_ticker bash_path                                                         
AAPL         stateoutput120.csv              1.673927               6.054788   
             stateoutput150.csv             -2.053609              -0.080618   
             stateoutput180.csv              4.960309               7.457115   
             stateoutput210.csv            -13.853165              -2.702211   
             stateoutput240.csv             -2.089565              -2.089565   
...                                               ...                    ...   
TSLA         stateoutput150.csv             -3.895730              -2.372135   
             stateoutput180.csv            -11.525128              -9.841231   
             stateoutput210.csv              0.604092              -4.457079   
             stateoutput270.csv             -8.694217             -10.969091   
             stateoutput60.csv              29.108333              20.556667   

                                 win_count  loss_count  
state_ticker bash_path                                  
AAPL         stateoutput120.csv          2           8  
             stateoutput150.csv          2          10  
             stateoutput180.csv          2          15  
             stateoutput210.csv          1          16  
             stateoutput240.csv          1           9  
...                                    ...         ...  
TSLA         stateoutput150.csv          1           9  
             stateoutput180.csv          1          13  
             stateoutput210.csv          3           8  
             stateoutput270.csv          2          17  
             stateoutput60.csv           1          12  

[478 rows x 9 columns]

In [670]:
portfolio_list = stock_tp_data.loc[stock_tp_data.estimate_with_thresh > -10].index.get_level_values(0).unique()

In [687]:
test_df = stock_tp_data.loc[stock_tp_data.index.get_level_values(0).isin(portfolio_list)]
x = test_df.loc[(test_df.estimate_with_thresh < -10)].groupby('state_ticker').count()
month_high_lost = x.loc[x.estimate_with_thresh > 2].index
new_portfolio_list = np.setdiff1d(portfolio_list, month_high_lost)

In [677]:
new_portfolio_list

array(['ADBE', 'ADP', 'AMD', 'AMGN', 'AMT', 'AMZN', 'AVGO', 'BMY', 'CHTR',
       'CMCSA', 'CME', 'COST', 'CSX', 'DE30', 'EA', 'GILD', 'HD', 'IBM',
       'INTU', 'JP225', 'KO', 'LIN', 'LMT', 'MA', 'MCD', 'MDLZ', 'MMM',
       'MO', 'MRK', 'MS', 'MSFT', 'NKE', 'PM', 'TMO', 'TSLA'],
      dtype=object)

In [699]:
stock_tp_data.loc[stock_tp_data.index.get_level_values(0).isin(new_portfolio_list)].sum()

state_close_estimate      975.979489
state_max_r              5880.564531
state_MA_r                814.682443
state_RSI_r               726.535107
last_r                    791.654915
estimate_with_thresh      757.917082
estimate_with_thresh2     726.722976
win_count                 355.000000
loss_count               1624.000000
dtype: float64

In [691]:
stock_tp_data.loc[stock_tp_data.index.get_level_values(0).isin(new_portfolio_list)].groupby('bash_path').sum()

,state_close_estimate,state_max_r,state_MA_r,state_RSI_r,last_r,estimate_with_thresh,estimate_with_thresh2,win_count,loss_count
bash_path,,,,,,,,,
stateoutput120.csv,67.010320,733.490306,35.511833,68.329262,42.518169,-24.817105,-44.422169,41,234
stateoutput150.csv,402.871446,1234.015344,500.310076,95.060175,497.769482,252.928590,180.202311,55,214
stateoutput180.csv,81.521129,745.247997,84.703261,85.521915,82.296092,-9.258017,-19.399602,55,204
stateoutput210.csv,117.525167,794.241566,79.947217,185.254151,77.452274,40.038968,48.447205,47,222
stateoutput240.csv,48.674513,474.296273,-6.207780,47.142834,-7.808037,-11.440605,3.426329,28,159
stateoutput270.csv,77.238673,724.529664,51.350412,71.471736,42.034867,5.430926,-22.568147,51,241
stateoutput60.csv,127.747117,800.462511,56.642939,102.317948,44.723780,12.057708,-10.392721,56,244
stateoutput90.csv,53.391123,374.280870,12.424486,71.437086,12.668286,-8.491284,9.043538,22,106


In [692]:
stock_tp_data.loc[stock_tp_data.index.get_level_values(0).isin(new_portfolio_list)].groupby('state_ticker').sum()

,state_close_estimate,state_max_r,state_MA_r,state_RSI_r,last_r,estimate_with_thresh,estimate_with_thresh2,win_count,loss_count
state_ticker,,,,,,,,,
ADBE,220.964351,399.441394,194.219489,57.508718,192.366481,169.593817,128.926647,11,59
ADP,21.583941,155.502693,12.236936,-2.457524,14.037927,7.788061,7.943657,11,49
AMD,11.401864,224.964498,-1.096410,38.392942,-8.922798,-20.887056,-22.177130,21,84
AMGN,72.121115,237.067871,56.421751,52.048203,57.008303,37.398224,40.105801,16,44
AMT,3.060792,121.414124,-15.636462,3.877702,-15.636462,-7.302456,-8.465490,11,43
AMZN,17.402300,242.385508,7.169030,13.171050,2.709005,-0.398647,-2.438424,10,82
AVGO,29.678328,192.376089,20.231226,29.334589,15.946556,8.878610,22.436574,10,48
BMY,46.115282,222.774265,13.434172,25.847930,29.852895,0.624626,-4.901496,18,54
CHTR,35.969377,276.665486,137.256788,40.030915,135.855632,21.052412,74.807741,12,49


In [682]:
stock_tp_data.loc[stock_tp_data.index.get_level_values(0) == 'AMGN']

state_close_estimate  state_max_r  \
state_ticker bash_path                                               
AMGN         stateoutput120.csv             23.451317    49.252260   
             stateoutput150.csv             39.468575    81.291867   
             stateoutput180.csv             -2.117927    14.941608   
             stateoutput210.csv             -9.000000    12.679169   
             stateoutput240.csv              9.437779    25.516933   
             stateoutput270.csv             -3.642188    13.776901   
             stateoutput60.csv               9.424121    25.733684   
             stateoutput90.csv               5.099438    13.875450   

                                 state_MA_r  state_RSI_r     last_r  \
state_ticker bash_path                                                
AMGN         stateoutput120.csv   20.066773    21.327219  20.200701   
             stateoutput150.csv   34.906407    30.189320  34.030450   
             stateoutput180.csv   -1.540654    -2.117927  -2.117927   
             stateoutput210.csv   -9.000000    -9.000000  -9.000000   
             stateoutput240.csv    5.459255     3.627033   5.872520   
             stateoutput270.csv   -3.428551    -3.642188  -3.642188   
             stateoutput60.csv     7.717897     9.424121   9.424121   
             stateoutput90.csv     2.240625     2.240625   2.240625   

                                 estimate_with_thresh  estimate_with_thresh2  \
state_ticker bash_path                                                         
AMGN         stateoutput120.csv             17.638724              22.184575   
             stateoutput150.csv             33.589601              37.060699   
             stateoutput180.csv             -1.053457              -1.045769   
             stateoutput210.csv             -9.000000              -2.641379   
             stateoutput240.csv              8.380042              10.130760   
             stateoutput270.csv             -3.821723              -2.366292   
             stateoutput60.csv               8.947216              15.157700   
             stateoutput90.csv               3.780432               5.475540   

                                 win_count  loss_count  
state_ticker bash_path                                  
AMGN         stateoutput120.csv          3           6  
             stateoutput150.csv          4           4  
             stateoutput180.csv          2           7  
             stateoutput210.csv          0           9  
             stateoutput240.csv          2           5  
             stateoutput270.csv          1           8  
             stateoutput60.csv           3           2  
             stateoutput90.csv           1           3

In [649]:
new_portfolio_list

array(['ADBE', 'ADP', 'AMD', 'AMGN', 'AMT', 'AMZN', 'AVGO', 'BMY', 'CHTR',
       'CMCSA', 'CME', 'COST', 'CSX', 'EA', 'GILD', 'HD', 'IBM', 'INTU',
       'KO', 'LIN', 'LMT', 'MA', 'MCD', 'MDLZ', 'MMM', 'MO', 'MRK', 'MS',
       'MSFT', 'NKE', 'PM', 'TMO', 'TSLA'], dtype=object)

In [705]:
spread_df = pd.read_csv('spread.csv')
spread_df.loc[spread_df.ticker.isin(new_portfolio_list)]
spread_df2 = pd.read_csv('spread2.csv')
spread_df['spred2'] = spread_df.ticker.apply(lambda x: spread_df2.spread.loc[spread_df2.ticker == x].iloc[0])
spread_df.head(50)

,Unnamed: 0,ticker,point,spread,deviation,spred2
0,0,AAPL,0.01,6,5,9
1,1,ABBV,0.01,7,5,7
2,2,ABT,0.01,7,5,7
3,3,ADBE,0.01,15,5,15
4,4,ADP,0.01,10,5,10
5,5,AMD,0.01,10,5,25
6,6,AMGN,0.01,11,5,11
7,7,AMT,0.01,10,5,10
8,8,AMZN,0.01,4,5,6
9,9,ATVI,0.01,7,5,7


# Stock_tp_data

In [616]:
def get_stock_data(tp_data):
    tp_data['estimate_with_thresh'] = tp_data.state_chain_new.apply(lambda x : get_return_with_max_estimate(x , 8 , 0.6) )
    tp_data['estimate_with_thresh2'] = tp_data.state_chain_new.apply(lambda x : x.max() * 0.75)
    tp_data['estimate_with_thresh2'].loc[tp_data['estimate_with_thresh2'] < 4] = -1
    tp_data.estimate_with_thresh2.loc[tp_data.estimate_with_thresh2 < 5] = -1
    stock_tp_data = tp_data[['state_ticker','state_close_estimate','state_max_r','state_MA_r','state_RSI_r','last_r', 'estimate_with_thresh', 'estimate_with_thresh2']].groupby('state_ticker').sum()
    stock_tp_data['win_count'] = tp_data[['state_ticker','state_close_estimate','state_max_r','state_MA_r','state_RSI_r','last_r', 'estimate_with_thresh', 'estimate_with_thresh2']].loc[tp_data.state_MA_r > 0].groupby('state_ticker').count().state_MA_r
    stock_tp_data['loss_count'] = tp_data[['state_ticker','state_close_estimate','state_max_r','state_MA_r','state_RSI_r','last_r' , 'estimate_with_thresh', 'estimate_with_thresh2']].loc[tp_data.state_MA_r < 0].groupby('state_ticker').count().state_MA_r
    stock_tp_data.win_count.fillna(0 , inplace = True)
    stock_tp_data['win_count'] = stock_tp_data['win_count'].astype(int) 

    return stock_tp_data

In [674]:
stock_tp_data = get_stock_data(concat_data)
stock_tp_data.sum()

state_close_estimate       157.072864
state_max_r              11269.896499
state_MA_r                -162.091560
state_RSI_r                 18.135929
last_r                    -235.822581
estimate_with_thresh      -222.719004
estimate_with_thresh2      232.272473
win_count                  712.000000
loss_count                4021.000000
dtype: float64

In [675]:
portfolio_list = stock_tp_data.loc[stock_tp_data.estimate_with_thresh > -10].index

In [615]:
path = '/Users/tranthong/Desktop/stock-app/state_output_new/state_output_new/stateoutput150.csv'
tp_data = preprocess_tp_data(path)
#tp_data['estimate_with_thresh'] = tp_data.state_chain_new.apply(lambda x : get_return_with_max_estimate(x , 15 , 0.6) )
stock_tp_data = get_stock_data(tp_data)
stock_tp_data.sum()

state_close_estimate      255.321413
state_max_r              2018.145404
state_MA_r                322.612335
state_RSI_r                 3.843986
last_r                    321.780874
estimate_with_thresh      152.794267
estimate_with_thresh2     274.091012
win_count                 103.000000
loss_count                579.000000
dtype: float64

In [529]:
def get_return_with_max_estimate(state_return_list, min_thresh = 5, cut_thresh = 0.7):
    c = np.array(state_return_list).copy()
    x_value = c[c > min_thresh]

    if len(x_value) > 0:
        x_value = x_value[0]
        x_index, = np.where(c == x_value)
        x = c[x_index[0]:]
        list_temp = [x[0] * cut_thresh]
        for i in range(1, len(x)):
            temp = x[0:i].max() * cut_thresh
            list_temp.append(temp)
        temp_df = pd.DataFrame()
        temp_df['a'] = x
        temp_df['b'] = list_temp
        # print(temp_df.iloc[50:])
        return temp_df.a.iloc[-1] if temp_df.loc[temp_df.a < temp_df.b].empty else  temp_df.loc[temp_df.a < temp_df.b].iloc[0].b
    else:
        lost_r = c[-1] if c[-1] > 0 else -1
        return lost_r


In [531]:
path = '/Users/tranthong/Desktop/stock-app/state_output_new/state_output_new/stateoutput240.csv'
tp_data = preprocess_tp_data(path)

tp_data['estimate_with_thresh'] = tp_data.state_chain_new.apply(lambda x : get_return_with_max_estimate(x , 8 , 0.6) )
a = tp_data.loc[(tp_data.estimate_with_thresh > 8)].iloc[0]
a.state_chain_new , a.estimate_with_thresh , a.state_chain_new.max() * 0.6

(array([ 1.49019608,  1.92156863,  1.25490196,  1.60784314,  1.29411765,
         1.09803922,  0.66666667,  0.54901961,  0.15686275,  0.35294118,
         0.62745098,  0.31372549,  0.78431373,  0.74509804,  1.33333333,
         1.56862745,  1.52941176,  1.76470588,  1.01960784,  1.33333333,
         1.64705882,  1.29411765,  0.94117647,  1.09803922,  0.70588235,
         0.74509804,  0.43137255,  0.2745098 ,  0.70588235,  0.43137255,
         0.43137255,  0.70588235,  0.78431373,  1.37254902,  1.84313725,
         1.68627451,  1.37254902,  1.49019608,  2.15686275,  2.74509804,
         3.05882353,  2.98039216,  2.94117647,  2.58823529,  2.62745098,
         2.58823529,  2.74509804,  2.35294118,  2.23529412,  2.35294118,
         2.35294118,  2.39215686,  2.70588235,  2.70588235,  2.78431373,
         2.15686275,  2.15686275,  2.62745098,  2.58823529,  2.35294118,
         2.19607843,  2.11764706,  2.35294118,  2.54901961,  2.07843137,
         2.35294118,  2.8627451 ,  2.54901961,  2.5

In [249]:
#tp_data = preprocess_tp_data('/Users/tranthong/Desktop/stock-app/state_output_new.csv')
def get_return_with_max_estimate(state_return_list, min_thresh = 12, cut_thresh = 0.66):
    c = np.array(state_return_list)
    x = c[c > min_thresh]
    if len(x) > 0:
        list_temp = [x[0]]
        for i in range(1, len(x)):
            temp = x[0:i].max() * cut_thresh
            list_temp.append(temp)
        temp_df = pd.DataFrame()
        temp_df['a'] = x
        temp_df['b'] = list_temp
        return temp_df.a.iloc[-1] if temp_df.loc[temp_df.a < temp_df.b].empty else  temp_df.loc[temp_df.a < temp_df.b].iloc[0].a
    else:
        lost_r = c[-1] if c[-1] > 0 else -1
        return lost_r

tp_data['new_estimate'] = tp_data.state_chain.apply(lambda x: get_return_with_max_estimate(x, 12, 0.66))
tp_data.new_estimate.sum()

86.98738404481048

In [152]:
tp_data.state_close_estimate.sum()

39.52315681719646

In [214]:
spread_df = pd.read_csv('/Users/tranthong/Desktop/stock-app/spread.csv')


In [212]:
tp_data = preprocess_tp_data('/Users/tranthong/Desktop/stock-app/state_output_new/state_output_new/stateoutput90.csv')

In [224]:
spread_df = pd.read_csv('/Users/tranthong/Desktop/stock-app/spread.csv')
tp_data['spread'] = tp_data.state_ticker.apply(lambda x: spread_df.spread.loc[spread_df.ticker == x].iloc[0])
tp_data['deviation'] = tp_data.state_ticker.apply(lambda x: spread_df.deviation.loc[spread_df.ticker == x].iloc[0])
tp_data['point'] = tp_data.state_ticker.apply(lambda x: spread_df.point.loc[spread_df.ticker == x].iloc[0])
tp_data['sum_gap'] = ((tp_data['spread'] + tp_data['deviation']) * tp_data['point']) / abs(tp_data['state_open'] - tp_data['state_stop_loss'] ) + 1
tp_data['state_chain_new'] = [np.array(y) / x for x,y in tp_data[['sum_gap' , 'state_chain']].to_numpy()]

In [244]:
x = tp_data.state_chain.iloc[0][0]

tp_data.loc[(tp_data.state_chain[0] == 1)]

KeyError: 'False: boolean label can not be used without a boolean index'

In [247]:
tp_data['state_chain_new'] = [np.array(y) / x for x,y in tp_data[['sum_gap' , 'state_chain']].to_numpy()]

In [250]:
tp_data.state_close_estimate.sum()

46.89175083298029

In [248]:
tp_data

,state_ticker,state_order,state_day_start,state_day_end,state_open,state_stop_loss,state_close_estimate,state_max_r,state_MA_r,state_RSI_r,last_r,state_chain,spread,deviation,point,sum_gap,state_chain_new
0,MRK,1,2022-11-09 14:55:00,2022-11-09 19:50:00,102.024,101.814,0.885714,7.504762,0.885714,0.885714,0.885714,"[1.5999999999999324, 3.1238095238094337, 3.552...",7,5,0.01,1.571429,"[1.0181818181817892, 1.9878787878787578, 2.260..."
1,MRK,1,2022-11-16 14:55:00,2022-11-16 15:00:00,99.946,99.656,-1.000000,-0.572414,-1.000000,-1.000000,-1.000000,"[-0.572413793103425, -0.882758620689637]",7,5,0.01,1.413793,"[-0.4048780487804681, -0.6243902439024211]"
2,MRK,2,2022-11-16 19:35:00,2022-11-16 19:55:00,99.898,100.028,-1.000000,0.984615,-1.000000,-1.000000,-1.000000,"[0.9846153846153123, 0.75384615384609, 0.52307...",7,5,0.01,1.923077,"[0.5119999999999807, 0.39199999999998075, 0.27..."
3,MRK,1,2022-11-25 15:30:00,2022-11-25 15:40:00,107.268,107.076,-1.000000,-0.458333,-1.000000,-1.000000,-1.000000,"[-0.770833333333284, -0.45833333333328397, -0....",7,5,0.01,1.625000,"[-0.4743589743589509, -0.2820512820512558, -0...."
4,MRK,1,2022-11-29 18:25:00,2022-11-29 18:55:00,108.816,108.592,-1.000000,0.241071,-1.000000,-1.000000,-1.000000,"[0.10714285714285941, -0.42857142857143765, -0...",7,5,0.01,1.535714,"[0.069767441860467, -0.279069767441868, -0.046..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,NFLX,2,2022-11-29 15:30:00,2022-11-30 11:15:00,281.168,283.248,-1.000000,2.354808,-1.000000,-1.000000,-1.000000,"[0.14326923076923273, 0.3596153846153828, 0.29...",24,5,0.01,1.139423,"[0.12573839662447417, 0.31561181434598967, 0.2..."
376,NFLX,1,2022-11-30 14:35:00,2022-11-30 14:40:00,283.605,281.780,-1.000000,-0.523288,-1.000000,-1.000000,-1.000000,"[-0.5232876712329024, -0.9178082191780942]",24,5,0.01,1.158904,"[-0.4515366430260266, -0.791962174940908]"
377,NFLX,1,2022-12-01 20:15:00,2022-12-01 20:20:00,318.462,317.632,-1.000000,-0.592771,-1.000000,-1.000000,-1.000000,"[-0.677108433734967, -0.5927710843373147]",24,5,0.01,1.349398,"[-0.501785714285732, -0.43928571428568647]"
378,NFLX,1,2022-12-02 20:25:00,2022-12-02 20:35:00,320.746,320.056,-1.000000,0.223188,-1.000000,-1.000000,-1.000000,"[0.22318840579701454, -0.09565217391308861, -0...",24,5,0.01,1.420290,"[0.1571428571427958, -0.06734693877554192, -0...."


In [157]:
path =  '/Users/tranthong/Desktop/stock-app/stock_data_M5/'
def get_mt5_raw_data_range(ticker, start, end):
    stock_dir = path + ticker +'_2022.csv'
    rates =  pd.read_csv(stock_dir, index_col =[0])
    rates['index'] = rates['index'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
    a = rates.loc[ (rates['index'] > start) & (rates['index'] < end )]
    a.reset_index(inplace = True ,drop = True)
    a['day_num'] = a.index
    return a

start = tp_data.loc[tp_data.state_close_estimate != tp_data.new_estimate].state_day_start.iloc[0]
end = tp_data.loc[tp_data.state_close_estimate != tp_data.new_estimate].state_day_end.iloc[0]
ticker = tp_data.loc[tp_data.state_close_estimate != tp_data.new_estimate].state_ticker.iloc[0]
open = tp_data.loc[tp_data.state_close_estimate != tp_data.new_estimate].state_open.iloc[0]
stoploss = tp_data.loc[tp_data.state_close_estimate != tp_data.new_estimate].state_stop_loss.iloc[0]
r = abs(open -stoploss)
data =  get_mt5_raw_data_range(ticker, start, end)

In [158]:
(data.Close - open) / r

0       0.292308
1       0.523077
2      -0.015385
3       0.446154
4       0.984615
         ...    
297    18.215385
298    18.215385
299    18.215385
300    18.061538
301    17.138462
Name: Close, Length: 302, dtype: float64